In [ ]:
'''
Copyright (C) 2022 Francesco Paparella, Pedro Velasquez

This file is part of "ACCESS IOT Stations".

"ACCESS IOT Stations" is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by the Free
Software Foundation, either version 3 of the License, or (at your option) any
later version.

"ACCESS IOT Stations" is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more
details.

You should have received a copy of the GNU General Public License along with
"ACCESS IOT Stations". If not, see <https://www.gnu.org/licenses/>.
'''

%matplotlib widget
import numpy as np
import pandas as pd
import textwrap
from pymongo import MongoClient
import time

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output

import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
# constant definitions
STATION = 'station1'
MONGO_IP = '10.224.83.51'
MONGO_PORT = 27017
DATABASE = 'stations'

In [ ]:
# connect to MongoDB
client = MongoClient(host=MONGO_IP, port=MONGO_PORT)
db = client[DATABASE]

In [ ]:
# initialize all buttons and dropdowns

# buttons to select station
max_station = db['stations_info'].count_documents({'config': True})

stations_dropdown = widgets.Dropdown(
    options=[(f"Station {str(i)}", i) for i in range(max_station)],
    description='Station:',
)

month_dropdown = widgets.Dropdown(
    options = [],
    description='YYYY_MM:')

sensor_dropdown = widgets.Dropdown(
    options = [],
    description='Sensor:')

measurement_dropdown = widgets.Dropdown(
    options = [],
    description='measurement:')

def update_month(*args):
    month_dropdown.options = db[f"station{stations_dropdown.value}"].distinct('month')
stations_dropdown.observe(update_month, 'value')
    
def update_sensor(*args):
    doc = db['stations_info'].find({'station_num': stations_dropdown.value})[0]
    sensors = doc['sensors']
    options = []
    for sensor in sensors.keys():
        if sensor == "gps":
            pass
        else:
            for s in range(sensors[sensor]):
                options.append(f"{sensor}{s}")
    sensor_dropdown.options = options
    
    
stations_dropdown.observe(update_sensor, 'value')
month_dropdown.observe(update_sensor, 'value')

def update_measurement(*args):
    options = []
    doc = db[f"station{stations_dropdown.value}"].find({'month':month_dropdown.value})[0]
    for attr in doc[f"{sensor_dropdown.value[0:-1]}"]:
        if attr != "type":
            options.append(attr)
    measurement_dropdown.options = options
    
if month_dropdown.value != None:    
    stations_dropdown.observe(update_measurement, 'value')
if sensor_dropdown.value != None:
    month_dropdown.observe(update_measurement, 'value')
sensor_dropdown.observe(update_measurement, 'value')
    
stations_box = widgets.Box(children=[stations_dropdown])
month_box = widgets.Box(children=[month_dropdown])
sensor_box = widgets.Box(children=[sensor_dropdown])
measurement_box = widgets.Box(children=[measurement_dropdown])

In [ ]:
def query(station_num, YYYY_MM, measurement):
    """
    station_num : the number of the station/collection to aggr from, i.e. "0"
    YYYY_MM : The Year and Month of the document to be querying, i.e. "2023_02"
    measurement : The measurement to return, in form "sensor.measurement.sensor_index", i.e. "particulate_matter.PM1count.1"

    returns a pandas dataframe containing the datetime and the queried measurement 
    """
    month = {'$match': {
                'month': '{}'.format(YYYY_MM)}
        }

    split_by_datetime = {'$unwind': {
                'path': '$gps.datetime', 
                'includeArrayIndex': 'index', 
                'preserveNullAndEmptyArrays': False}
        }

    exclude_null = {'$match': {
                'gps.datetime': {
                    '$ne': None}}
        }

    targets = {'$project': {
                '_id': 0, 
                'datetime': '$gps.datetime', 
                measurement.split(".")[1] + measurement.split(".")[2]: {
                '$arrayElemAt': [f"${measurement}", '$index']}
        }
        }

    aggr = db[f"station{station_num}"].aggregate([
        month, split_by_datetime, exclude_null, targets])
    data = pd.DataFrame(list(aggr))
    return data



In [ ]:
display(stations_box)
display(month_box)
display(sensor_box)
display(measurement_box)